In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import AsinhStretch, ImageNormalize
from scipy.ndimage import gaussian_filter
#import sys
#sys.path.insert(0, '/Users/smericks/Desktop/StrongLensing/darkenergy-from-lagn/')
#from lens_catalog import OM10LensCatalog

from lenstronomy.ImSim.image_model import ImageModel
from lenstronomy.Data.pixel_grid import PixelGrid
from lenstronomy.Data.psf import PSF
from lenstronomy.LightModel.light_model import LightModel
from lenstronomy.Util.param_util import phi_q2_ellipticity, ellipticity2phi_q

## Construct a simple test case for $v_{rms}$ calculations ##

In [ ]:
# make a lens catalog with the simple test case 

num_lenses = 11

test_case_df = pd.DataFrame()
# Lens Light
test_case_df['lens_light_parameters_e1'] = np.zeros(num_lenses) # spherical
test_case_df['lens_light_parameters_e2'] = np.zeros(num_lenses)
test_case_df['lens_light_parameters_n_sersic'] = np.ones(num_lenses)*4. #assuming n_sersic=4
test_case_df['lens_light_parameters_R_sersic'] = np.ones(num_lenses)*0.5
# Lens Mass
test_case_df['main_deflector_parameters_e1'] = np.zeros(num_lenses) # spherical
test_case_df['main_deflector_parameters_e2'] = np.zeros(num_lenses)
test_case_df['main_deflector_parameters_theta_E'] = np.ones(num_lenses)*1. #assuming theta_E=1
test_case_df['main_deflector_parameters_gamma'] = np.asarray([1.5,1.6,1.7,1.8,1.9,2.0,2.1,2.2,2.3,2.4,2.5])

test_case_df.to_csv('test_case.csv')

## Investigate $v_{rms}$ maps from SKiNN ##

'test_case_skinn_maps.npy' stores SKiNN generated $v_{rms}$ maps.

Notes:
- These maps are on a 551x551 grid, with a pixel scale of 0.02"
- b_ani = 0.
- inclination = 90.

Load lens catalog & corresponding images

In [ ]:
#test_catalog = pd.read_csv('../MassModels/om10_sample/om10_venkatraman_erickson24.csv')
test_catalog = pd.read_csv('test_case.csv')
vrms_maps = np.load('test_case_skinn_maps.npy')

Calculate aperture-integrated, luminosity weighted $v_{rms}$

from SKiNN (Gomer '23):
$v_\mathrm{rms} = \sqrt{v_\mathrm{rot}^2 + \sigma_\mathrm{v}^2}$

NOTE: SKiNN $v_\mathrm{rms}$ are produced at $z_\mathrm{lens}=0.5$ and $z_\mathrm{lens}=2.0$ (can be re-scaled for other redshifts)

to calculate the final product (modified from Huang '25): 
$[\langle v_\mathrm{rms}^2 \rangle]_\mathcal{A} = \frac{\int_{\mathcal{A}}\left[\Sigma(x',y') \langle v_\mathrm{rms}^2 \rangle * \mathcal{P}\right] \mathrm{d}x'\mathrm{d}y'}{\int_{\mathcal{A}}\left[\Sigma(x',y') * \mathcal{P}\right] \mathrm{d}x'\mathrm{d}y'}$

In [ ]:
vrms_maps = np.load('test_case_skinn_maps.npy')
# step 1: get a light profile (in surface brightness) on the same grid ()
# step 2: multiply light profile by v_rms map
# step 3: convolve with PSF
# step 4: sum over aperture
# step 5: repeat w/out multiplying in v_rms, and divide out to do luminosity-weighting

NUMPIX = 551

# set to match v_rms map from SKiNN! (551x551 at 0.02" resolution)
pixel_grid = PixelGrid(
        nx=NUMPIX,ny=NUMPIX,
        transform_pix2angle=np.asarray([[0.02,0.],[0.,0.02]]),
        ra_at_xy_0=-0.02*275.,dec_at_xy_0=-0.02*275.
)

# PSF is simple, fwhm=0.5"
psf = PSF(
    psf_type='GAUSSIAN',fwhm=0.5
)

# LightModel
light_model = LightModel(['SERSIC_ELLIPSE'])

lens_galaxy_model = ImageModel(
    data_class=pixel_grid,
    psf_class=psf,
    lens_light_model_class=light_model  
)


fig,axs = plt.subplots(2,5,dpi=200,figsize=(6,2.5))
ll_norm = None
vrms_norm = None

for i in range(0,5):

    # get dict of params for this lens
    lens_params = test_catalog.loc[i]

    # need to remove orientation angle to be consistent with SKiNN... 
    # 1) convert e1,e2 to q,phi 2) set phi=0, convert back
    phi, q = ellipticity2phi_q(
        e1=lens_params['lens_light_parameters_e1'],
        e2=lens_params['lens_light_parameters_e1']
    )
    e1, e2 = phi_q2_ellipticity(phi=0.,q=q)

    #"amp", "R_sersic", "n_sersic", "e1", "e2", "center_x", "center_y"
    # amp shouldn't matter if we weight out luminosity ... ?
    kwargs_lens_light = [{
        'amp':10.,
        'n_sersic':lens_params['lens_light_parameters_n_sersic'],
        'R_sersic':lens_params['lens_light_parameters_R_sersic'],
        'e1':e1,
        'e2':e2,
        'center_x':0.,
        'center_y':0.
    }]

    # Sigma(x,y)
    lens_sb = lens_galaxy_model.lens_surface_brightness(kwargs_lens_light,unconvolved=True)
    # v_rms(x,y)
    v_rms = vrms_maps[i]
    # TODO: convolve with PSF!
    # FWHM = 2sqrt(2*ln(2)) * sigma
    psf_fwhm = 0.5/0.02 # in pixels (fwhm=0.5", divided by pixel scale 0.02")
    sigma = psf_fwhm / (2*np.sqrt(2*np.log(2)))

    numerator = gaussian_filter((lens_sb * (v_rms**2)),sigma=sigma,mode='nearest')
    denominator = gaussian_filter(lens_sb,sigma=sigma,mode='nearest')

    if ll_norm is None:
        ll_norm = ImageNormalize(denominator[min_val:max_val,min_val:max_val])
    # plot surface brightness convolved with PSF
    plt_im = axs[0,i%5].matshow(denominator[min_val:max_val,min_val:max_val],norm=ll_norm)

    # sum-up within a random aperture
    size_box = 201
    center_pix = (NUMPIX-1)/2 + 1
    min_val = int(center_pix-1 - (size_box-1)/2)
    max_val = int(center_pix + (size_box-1)/2)
    #225 -> 326

    # TODO: currently summing over a square aperture...

    R_ap = 2./0.02 # 2" aperture
    R_inner = 0.5/0.02 # mask center 0.5" pixels
    # create grid and evaluate whether Radius is inside or outside R_aperture
    center_pix_vals = np.arange(-NUMPIX/2 +0.5,NUMPIX/2 +0.5, 1.)
    x_grid, y_grid = np.meshgrid(center_pix_vals, center_pix_vals)
    R_grid = np.sqrt(x_grid**2 + y_grid**2)
    # zeroing out these #s is equivalent to ignoring them in the averaging
    numerator[R_grid>R_ap] = 0.
    denominator[R_grid>R_ap] = 0.
    numerator[R_grid<R_inner] = 0.
    denominator[R_grid<R_inner] = 0.
    v_rms_weighted = np.sum(numerator) / np.sum(denominator)

    #print(v_rms_weighted)

    # let's plot things to double check what's going on
    vrms_norm = ImageNormalize(vrms_maps[2][min_val:max_val,min_val:max_val])
    if vrms_norm is None:
        vrms_norm = ImageNormalize(vrms_maps[i][min_val:max_val,min_val:max_val])
    # plot v_rms map
    vrms_maps[i][R_grid>R_ap] = 0.
    vrms_maps[i][R_grid<R_inner] = 0.
    v_rms_box = vrms_maps[i][min_val:max_val,min_val:max_val]
    #plt_vrms = axs[1,i%5].matshow(vrms_maps[i][min_val:max_val,min_val:max_val],norm=vrms_norm)
    axs[0,i%5].set_title('$\gamma_{lens} = %.1f$'%(lens_params['main_deflector_parameters_gamma']),fontsize=5)
    axs[1,i%5].set_title('[sqrt{<$v_{rms}^2$>}]$_{A}$ = %.2f'%(np.sqrt(v_rms_weighted)),fontsize=4)
    axs[0,i%5].set_xticks([])
    axs[0,i%5].set_yticks([])

    # TODO: plot a circle with radius 2"
    phi = np.arange(0,2*np.pi,0.01)
    R = 2./0.02
    x = R*np.cos(phi) + size_box/2 -0.5
    y = R*np.sin(phi) + size_box/2 -0.5
    axs[1,i%5].scatter(x, y, color='black', s=1, linewidth=0.1)
    R = 0.5/0.02
    x = R*np.cos(phi) + size_box/2 -0.5
    y = R*np.sin(phi) + size_box/2 -0.5
    axs[1,i%5].scatter(x, y, color='black', s=1, linewidth=0.1)
    plt_vrms = axs[1,i%5].matshow(v_rms[min_val:max_val,min_val:max_val],norm=vrms_norm)
    #axs[1,i%5].scatter(x_grid,y_grid,color='red',s=0.5,linewidth=0.001)
    #axs[1,i%5].scatter(0,0,color='red',s=20)
    axs[1,i%5].set_xticks([])
    axs[1,i%5].set_yticks([])
    if i == 0:
        axs[0,i].set_ylabel('$\Sigma(x,y)$ * $P$')
        axs[1,i].set_ylabel('$v_{rms}$')

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.82, 0.53, 0.015, 0.35])
fig.colorbar(plt_im, cax=cbar_ax).ax.tick_params(labelsize=6)
cbar_ax = fig.add_axes([0.82, 0.11, 0.015, 0.35])
fig.colorbar(plt_vrms, cax=cbar_ax).ax.tick_params(labelsize=6)

# TODO: try masking out the center regions a well ("shell") aperture!

## Compare to lenstronomy ##

In [ ]:
from lenstronomy.Analysis.kinematics_api import KinematicsAPI

# circular aperture
aperture_type = 'shell'
kwargs_aperture = {'aperture_type': aperture_type, 'r_in': 0.5, 'r_out': 2.,'center_ra': 0, 'center_dec': 0}

# psf with fwhm=0.5
kwargs_seeing = {'psf_type': 'GAUSSIAN', 'fwhm': 0.5}

anisotropy_model = 'const'
kwargs_anisotropy = {'beta': 0.}

kwargs_lens = [{
    'theta_E':1., 
    'gamma':1.5, 
    "center_x":0., 
    "center_y":0.
}]

kwargs_lens_light = [{
    'amp': 10.,
    'R_sersic': 0.5,
    'n_sersic': 4.,
    'center_x': 0.,
    'center_y': 0.,
}]

kwargs_numerics_galkin = { 
                          'interpol_grid_num': 1000,  # numerical interpolation, should converge -> infinity
                          'log_integration': True,  # log or linear interpolation of surface brightness and mass models
                           'max_integrate': 100, 'min_integrate': 0.001}  # lower/upper bound of numerical integrals


kwargs_model = {
    'lens_model_list':['SPP'],
    'lens_light_model_list':['SERSIC']
}

kinematicsAPI = KinematicsAPI(0.5, 2., kwargs_model, kwargs_aperture, kwargs_seeing, anisotropy_model, 
    kwargs_numerics_galkin=kwargs_numerics_galkin, lens_model_kinematics_bool=[True, False],
    sampling_number=5000,MGE_light=True)  # numerical ray-shooting, should converge -> infinity)


vel_disp_numerical = kinematicsAPI.velocity_dispersion(kwargs_lens, kwargs_lens_light, kwargs_anisotropy, r_eff=0.5,theta_E=1.)
theta_E = kwargs_lens[0]['theta_E']
gamma = kwargs_lens[0]['gamma']
#vel_disp_analytic = kinematicsAPI.velocity_dispersion_analytical(theta_E, gamma, r_ani=r_ani, r_eff=r_eff)
print(vel_disp_numerical, ' velocity dispersion in km/s')